## Segmenting and Clustering Neighborhoods in Toronto 
PART_ONE<br><br>
1. Start by creating a new Notebook for this assignment.

2. Use the Notebook to build the code to scrape the following Wikipedia page,
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data 
that is in the table of postal codes and to transform the data into a pandas dataframe

 

get environment done

In [14]:
conda install html5lib

Solving environment: done

## Package Plan ##

  environment location: /home/pa/anaconda3

  added / updated specs:
    - html5lib


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2021.5.3~ --> pkgs/main::ca-certificates-2021.7.5-h06a4308_1

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge::certifi-2021.5.30-py38h5~ --> pkgs/main::certifi-2021.5.30-py38h06a4308_0
  conda              conda-forge::conda-4.10.3-py38h578d9b~ --> pkgs/main::conda-4.10.3-py38h06a4308_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [2]:
#import packages
import numpy as np 
import pandas as pd
import json 
import requests 
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import lxml

#STEP_ONE: get data from the wiki url
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
# use BeautifulSoup to open it, define the referenced source list as canada_list
canada_list = BeautifulSoup(source, 'lxml') 
#creat the target dataframe columns and give them a name
toronto_list = pd.DataFrame(columns=['Postalcode','Borough', 'Neighborhood'])

#STEP_TWO: The most import loop process to get data from the web
#inspect structure: div(class)->table(class)->  {thead()->tr()->th(class)} OR {tbody()->tr()->td(table elements)}
content = canada_list.find('div', class_='mw-parser-output')
table = content.table.tbody #can't be writen as the same row from above row
postcode = 0
borough = 0
neighborhood = 0

#loop each elements into the table, which located at {tbody()->tr()->td(table elements)}
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:  #the first element followed by <td> as postcode
            postcode = td.text
            i = i + 1
        elif i == 1: #the second element followed by <td> as borough
            borough = td.text
            i = i + 1
        elif i == 2: #the third element followed by <td> as neighborhood
            neighborhood = td.text.strip('\n').replace(']','') #need to gather neighbors in one cell
    toronto_list = toronto_list.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

#STEP_THREE: Remove unwanted values
# remove Not assigned borough, both 'not assigned or = 0'
toronto_list = toronto_list[toronto_list.Borough!='Not assigned']
toronto_list = toronto_list[toronto_list.Borough!= 0]
toronto_list.reset_index(drop = True, inplace = True)

#
i = 0
for i in range(0,toronto_list.shape[0]):
    if toronto_list.iloc[i][2] == 'Not assigned':
        toronto_list.iloc[i][2] = toronto_list.iloc[i][1]
        i = i+1
        
#print the dataframe
df = toronto_list.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M9AEtobicoke(Islington Avenue)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M9BEtobicoke(West Deane Park / Princess Garden...
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M9ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M9GNot assigned


In [3]:
df.tail()

,Postalcode,Borough,Neighborhood
15,\nM1VScarborough(Milliken / Agincourt North / ...,\nM2VNot assigned\n\n,M9VEtobicoke(South Steeles / Silverstone / Hum...
16,\nM1WScarborough(Steeles West / L'Amoreaux Wes...,\nM2WNot assigned\n\n,M9WEtobicokeNorthwest(Clairville / Humberwood ...
17,\nM1XScarborough(Upper Rouge)\n\n,\nM2XNot assigned\n\n,M9XNot assigned
18,\nM1YNot assigned\n\n,\nM2YNot assigned\n\n,M9YNot assigned
19,\nM1ZNot assigned\n\n,\nM2ZNot assigned\n\n,M9ZNot assigned


In [4]:
df.shape

(20, 3)

## PART_TWO
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:
Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:

In [5]:
# Use the csv to inport lat and lng.
# Redirected errors, instead downloading to file stored on local harddrive.
df_coord = pd.read_csv("Geospatial_Coordinates_v2.csv")

df_coord.rename(columns={'Postal Code':'Postalcode'}, inplace=True)

#join two table together on Postalcode
df = pd.merge(df, df_coord, how='inner', on = 'Postalcode')

#save the file incase you change the content by chance.
#df.to_csv('toronto_df.csv')

df_Geo_Coordinates_Burroughs_etc = pd.read_csv('Geo_Coordinates_Burroughs_etc.csv')
df_Geo_Coordinates_Burroughs_etc

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
2,M1R,Scarborough,"Wexford, Maryvale",43.750071,-79.295849
3,M2H,North York,Hillcrest Village,43.803762,-79.363452
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
5,M4G,East York,Leaside,43.709060,-79.363452
6,M4M,East Toronto,Studio District,43.659526,-79.340923
7,M5A,Downtown Toronto,"regent Park, Harbourfront",43.654260,-79.360636
8,M5G,Downtown Toronto,Central Bay street,43.657952,-79.387383
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands",43.628947,-79.394420


## PART_THREE
Explore and cluster the neighborhoods in Toronto. <br>I decided to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data.
<br>

### I'll follow and replicate the same analysis we did to the New York City data
#### Explore Toronto surroundings
#### Explore the borough of Etobicoke
<br>
get environment done

In [6]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /home/pa/anaconda3

  added / updated specs:
    - geopy


The following packages will be UPDATED:

  conda              pkgs/main::conda-4.10.3-py38h06a4308_0 --> conda-forge::conda-4.10.3-py38h578d9bd_1

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2021.7.5-h~ --> conda-forge::ca-certificates-2021.5.30-ha878542_0
  certifi            pkgs/main::certifi-2021.5.30-py38h06a~ --> conda-forge::certifi-2021.5.30-py38h578d9bd_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

In [8]:
# Step 1 Use geopy library to get the latitude and longitude values of Toronto.
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON are 43.6534817, -79.3839347.


In [9]:
# Step_2 create map of Toronto using latitude and longitude values
neighborhoods = df[['Borough','Neighborhood','Latitude','Longitude']]

# Step_3 create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='#66bd55',
        fill=True,
        fill_color='#5872c4',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [10]:
#Step_4 I chose Etobicoke in Toronto for analysis.

etobicoke_data = neighborhoods[neighborhoods['Borough'] == 'Etobicoke'].reset_index(drop=True)
etobicoke_data.head()

,Borough,Neighborhood,Latitude,Longitude


In [11]:
#Step_5 get the lat and lng for Etobicoke.
address = 'Etobicoke, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

#print('The geograpical coordinate of Etobicoke.
print('The geograpical coordinate of Etobicoke is: {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke is: 43.6435559, -79.5656326.


In [12]:
#Step_6 create map of Etobicoke using latitude and longitude values

map_etobicoke = folium.Map(location=[latitude, longitude], zoom_start=12.5)

for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='#E42222',
        fill=True,
        fill_color='#CB9D5B',
        fill_opacity=0.7,
        
        parse_html=False).add_to(map_etobicoke)  
    
map_etobicoke

In [13]:
#Step_7 Use  Foursquare to explore the surrounds of Toronto
#CLIENT_ID = 'USER SECRET - ENCRYPTED KEY'
#CLIENT_SECRET = 'USER SECRET - ENCRYPTED KEY'

VERSION = '20160604'

# read to csv to store locally to use instead of giving my login away.
#df.to_csv('toronto_df.csv')


# cleansed dataset from foursquare with content and postcodes combined to read from local copy of FourSquare.
#cleansedDataset_foursquareContent_Postcodes = pd.read_csv('cleansedDataset_foursquareContent_Postcodes.csv')
etobicoke_data = pd.read_csv('cleansedDataset_foursquareContent_Postcodes.csv')

#get neighbor's name
etobicoke_data.loc[0, 'Neighborhood']

#Get neighbor's lat and long
neighborhood_latitude = etobicoke_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = etobicoke_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = etobicoke_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

KeyError: 'Neighborhood'

In [ ]:
#Step_9
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

In [ ]:
from IPython.display import Image
Image(url = 'https://cdn1.imggmi.com/uploads/2019/10/9/9939d6c7a5933d5b90bc64892efb45069-full.png')

In [ ]:
#Step_10 get venues near Etobicoke.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = df_cleansedDataset_foursquareContent_Postcodes
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
etobicoke_venues = getNearbyVenues
                             

In [ ]:
#Step_12 Check the size of venues
print(etobicoke_venues.shape)
etobicoke_venues.head()

In [ ]:
#Step_13 How many venues each neighbor has
etobicoke_venues.groupby('Neighborhood').count()

In [ ]:
#Step_13 how many unique catetories in those venues
print('There are {} uniques categories.'.format(len(etobicoke_venues['Venue Category'].unique())))

In [ ]:
#Step_14 Analize each neighbor
# one hot encoding
etobicoke_onehot = pd.get_dummies(etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etobicoke_onehot['Neighborhood'] = etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [etobicoke_onehot.columns[-1]] + list(etobicoke_onehot.columns[:-1])
etobicoke_onehot = etobicoke_onehot[fixed_columns]

etobicoke_onehot.head()

In [ ]:
etobicoke_onehot.shape

In [ ]:
#Step_15 Group neighbor and confirm Size
etobicoke_grouped = etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
etobicoke_grouped.shape

In [ ]:
#Step_16 print neighbor and it's top 5 venues
num_top_venues = 5

for hood in etobicoke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = etobicoke_grouped[etobicoke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
#Step_17 Let's put the data into a dataframe and get 10 venues for each neighborhood this time
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = etobicoke_grouped['Neighborhood']

for ind in np.arange(etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
#Step_18 Setting clusters to five cluster groups of neighborhood.
# set number of clusters
kclusters = 5

etobicoke_grouped_clustering = etobicoke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
#Step_19 new data for the following graph
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

etobicoke_merged = etobicoke_data

# merge etobicoke_grouped with etobicoke_data to add latitude/longitude for each neighborhood
etobicoke_merged = etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

etobicoke_merged.head() # check the last columns!

In [ ]:
etobicoke_merged.dtypes

In [ ]:
#This role is for when you need to use the cluster for data, but color=rainbow[cluster-1] can't take float, then you have to convert the Cluster Labels into int.
etobicoke_merged= etobicoke_merged.fillna(0)
etobicoke_merged[['Cluster Labels']] = etobicoke_merged[['Cluster Labels']].astype("int")

In [ ]:
#Step_20 Last step, view the clustered data
# create map
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(etobicoke_merged['Latitude'], etobicoke_merged['Longitude'], etobicoke_merged['Neighborhood'], etobicoke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters